In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_df
from asteroid_element import make_sim_asteroids, get_ast_ref_elts, add_asteroid_elts, get_asteroids
from asteroid_integrate import integrate_ast
from asteroid_integration_test import get_diff_by_date, get_diff_by_ast
from db_config import db_engine
from db_utils import sp2df

In [3]:
# Typing
from typing import Tuple

In [4]:
epoch = 59000
n0 = 0
n1 = 100
elts = get_ast_ref_elts(epoch=epoch, n0=n0, n1=n1, missing=False)

In [5]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,M,BodyID,BodyName
0,1,84960000,Ceres,59000.0,2.767657,0.077557,0.184806,1.401272,1.286860,2.839411,1000001,SB.Ceres
1,2,84960000,Pallas,59000.0,2.773841,0.229972,0.607949,3.019851,5.414053,2.530303,1000002,SB.Pallas
2,3,84960000,Juno,59000.0,2.668285,0.256936,0.226736,2.964468,4.329572,2.189260,1000003,SB.Juno
3,4,84960000,Vesta,59000.0,2.362014,0.088516,0.124650,1.811810,2.633265,3.566190,1000004,SB.Vesta
4,5,84960000,Astraea,59000.0,2.574037,0.190913,0.093679,2.470881,6.259596,0.311477,1000005,SB.Astraea
...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,84960000,Arethusa,59000.0,3.064051,0.152789,0.226817,4.241497,2.684048,2.578014,1000095,SB.Arethusa
95,96,84960000,Aegle,59000.0,3.048360,0.141178,0.278942,5.611058,3.638345,3.115040,1000096,SB.Aegle
96,97,84960000,Klotho,59000.0,2.669021,0.257298,0.205558,2.785849,4.689231,0.723346,1000097,SB.Klotho
97,98,84960000,Ianthe,59000.0,2.686497,0.187233,0.271867,6.176797,2.768649,3.757108,1000098,SB.Ianthe


In [6]:
sim = make_sim_asteroids(epoch=epoch, n0=n0, n1=n1, missing=True)

In [7]:
sim.body_ids

array([10, 1, 2, 399, 301, 4, 5, 6, 7, 8, 9], dtype=object)

In [8]:
# sim.body_namesm

In [9]:
mjd0=58900
mjd1=59100
steps_per_day=1
progbar=True

In [10]:
df_vec, df_elt = integrate_ast(sim=sim, mjd0=mjd0, mjd1=mjd1, interval=4, progbar=progbar)


********************************************************************************
Integrating 0 asteroids in 0000000-0000000 from 58900 to 58900...


|          | 0/0 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
df = get_diff_by_ast()

In [12]:
df

,AsteroidID,dq,dq_rel
0,1,3.096302e-06,1.118135e-06
1,2,1.074489e-06,3.873648e-07
2,3,5.764787e-07,2.160484e-07
3,4,1.990539e-06,8.429421e-07
4,5,7.173841e-07,2.787000e-07
...,...,...,...
359,1036,7.778635e-06,2.919114e-06
360,1093,2.240079e-06,7.152727e-07
361,1107,2.876499e-06,9.038379e-07
362,1171,1.211640e-06,3.818223e-07


In [17]:
(84971520-84965760)/60/24

4.0